In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import json
import speech_recognition as sr
import os
import pandas as pd

In [2]:
def transcrever_captcha():
    r = sr.Recognizer()
    audio_file = sr.AudioFile('pgAudio.wav')
    
    with audio_file as source:
        audio = r.record(source)
    
    try:
        captcha_str = r.recognize_google(audio,language="pt-BR").replace(" ","")
        return captcha_str
    
    except:
        return 'XXXXXXX' 

In [3]:
def converter_mp3_to_wav():
    os.system("rm *.wav")
    os.system("ffmpeg -i pgAudio.mp3 pgAudio.wav")

In [4]:
def executar_scrapping_moz( ano,mes,dia,caderno,protocolo ):
        
    url = 'http://processos.prefeitura.sp.gov.br/Forms/consultarProcessos.aspx'
    urlAudioCaptcha = 'http://processos.prefeitura.sp.gov.br/pgAudio.ashx?'
    urlValidaCaptcha = 'http://processos.prefeitura.sp.gov.br/ValidaCaptchaHandler.ashx?textoCaptcha='
    urlCarregarPorNumero = 'http://processos.prefeitura.sp.gov.br/Forms/consultarProcessos.aspx/CarregaDadosPorNumero'
    urlDadosLocalizacao = 'http://processos.prefeitura.sp.gov.br/Forms/consultarProcessos.aspx/CarregaDadosLocalizacaoAtual'
    
    driver = webdriver.Firefox()
    driver.get(url)
    
    cookies_list = driver.get_cookies()
    cookies_dict = {}
    
    for cookie in cookies_list:
        cookies_dict[cookie['name']] = cookie['value']

    session_id = cookies_dict['ASP.NET_SessionId']
    
    driver.execute_script("TocarAudio()")
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    captchaMp3 = urlAudioCaptcha + str(soup.audio).split('?')[1].split('\"')[0]
    
    headers_audio = {
                    "Host":"processos.prefeitura.sp.gov.br"
                    ,"User-Agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:65.0) Gecko/20100101 Firefox/65.0"
                    ,"Accept":"audio/webm,audio/ogg,audio/wav,audio/*;q=0.9,application/ogg;q=0.7,video/*;q=0.6,*/*;q=0.5"
                    ,"Accept-Language":"en-US,en;q=0.5"
                    ,"Referer":"http://processos.prefeitura.sp.gov.br/Forms/consultarProcessos.aspx"
                    ,"Range":"bytes=0-"
                    ,"Connection": "keep-alive"
                    ,"Cookie": "ASP.NET_SessionId="+session_id
                }
        
    r = requests.get(captchaMp3, headers=headers_audio, stream = True)
    
    handle = open("pgAudio.mp3", "wb")
    
    for chunk in r.iter_content(chunk_size=512):
        if chunk:  # filter out keep-alive new chunks
            handle.write(chunk)
            
    converter_mp3_to_wav()
    
    captcha_str = transcrever_captcha()
    
    if(len(captcha_str) != 4):
        driver.close()
        return False    
    
    #input str no campo busca protocolo
    captcha_in = driver.find_element_by_id("ContentPlaceHolder1_txtNumero")
    captcha_in.clear()
    captcha_in.send_keys(protocolo)
    
    #input str no captcha
    captcha_in = driver.find_element_by_id("txtCaptcha")
    captcha_in.clear()
    captcha_in.send_keys(captcha_str)
    
    driver.find_element_by_id("ContentPlaceHolder1_btnConsultar").click()
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    if "modal fade in" in str(soup.find("div", {"id": "alert"})): 
        driver.close()
        return False
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    
    assunto = soup.find("label", {"id": "lblAssunto"}).text 
    assunto = verificar(assunto)
    
    subassunto = soup.find("label", {"id": "lblSubassunto"}).text
    subassunto = verificar(subassunto)
    
    motivoAutuacao = soup.find("label", {"id": "lblMotivoAutuacao"}).text
    motivoAutuacao = verificar(motivoAutuacao)
    
    dtAutuacao = soup.find("label", {"id": "lblDatAutuacao"}).text
    dtAutuacao = verificar(dtAutuacao)
    
    interessado = soup.find("label", {"id": "lblInteressado"}).text
    interessado = verificar(interessado)
    
    sql = soup.find("label", {"id": "lblSql"}).text
    sql = verificar(sql)
    
    sigla = soup.find("label", {"id": "lblSigla"}).text
    sigla = verificar(sigla)
    
    nomeCompleto = soup.find("label", {"id": "lblNome"}).text
    nomeCompleto = verificar(nomeCompleto)
    
    orgaoSupervisor = soup.find("label", {"id": "lblOrgaoSuperior"}).text
    orgaoSupervisor = verificar(orgaoSupervisor)
    
    dtRecebimento = soup.find("label", {"id": "lblDataRecebimento"}).text
    dtRecebimento = verificar(dtRecebimento)
    
    funcao = soup.find("label", {"id": "lblFuncao"}).text
    funcao = verificar(funcao)
    
    responsavelLegal = soup.find("label", {"id": "lblResponsavel"}).text
    responsavelLegal = verificar(responsavelLegal)
    
    Cargo = soup.find("label", {"id": "lblCargo"}).text
    Cargo = verificar(Cargo)
    
    Logradouro = soup.find("label", {"id": "lblLogradouro"}).text
    Logradouro = verificar(Logradouro)
    
    Complemento = soup.find("label", {"id": "lblComplemento"}).text
    Complemento = verificar(Complemento)
    
    Bairro = soup.find("label", {"id": "lblBairro"}).text
    Bairro = verificar(Bairro)
    
    Cep = soup.find("label", {"id": "lblCEP"}).text
    Cep = verificar(Cep)
    
    Telefone = soup.find("label", {"id": "lblTelefone"}).text
    Telefone = verificar(Telefone)
    
    Ramal = soup.find("label", {"id": "lblRamal"}).text
    Ramal = verificar(Ramal)
    
    HrAtendimento = soup.find("label", {"id": "lblHorario"}).text
    HrAtendimento = verificar(HrAtendimento)
    
    EndEletronico = soup.find("label", {"id": "lblEnderecoEletronico"}).text
    EndEletronico = verificar(EndEletronico)

    linha = ano + ';' + mes + ';' + dia + ';' + caderno + ';' + protocolo + ';' + assunto + ';' + subassunto + ';' + motivoAutuacao + ';' + dtAutuacao + ';' + interessado + ';' + sql + ';' + sigla + ';' + nomeCompleto + ';' + orgaoSupervisor + ';' + dtRecebimento + ';' + funcao + ';' + responsavelLegal + ';' + Cargo + ';' + Logradouro + ';' + Complemento + ';' + Bairro + ';' + Cep + ';' + Telefone + ';' + Ramal + ';' + HrAtendimento + ';' + EndEletronico
    
    salvar('apresentacao','csv',linha)
    
    driver.close()

In [5]:
def verificar(valor):
    return valor if bool(valor.strip()) else 'nao_contem'

In [7]:
def executar():
    caminho = [os.path.join("DOWNLOAD/PROCESSADO/", nome) for nome in os.listdir("DOWNLOAD/PROCESSADO/")]
    arquivos = [arq for arq in caminho if os.path.isfile(arq)]

    for arq in arquivos:

        with open(arq, 'r') as fileopen:
            csv = pd.read_csv(arq,sep=';')
            #print csv
            for index, row in csv.iterrows():
                ano = row[0].split('_')[0]
                mes = row[0].split('_')[1]
                dia = row[0].split('_')[2]
                caderno = row[1]
                continua = (row[2] > 0) if row[2] else 'erro'

                if(continua):
                    loop = False
                    while loop is False:
                        loop = executar_scrapping_moz(ano,mes,dia,caderno,row[2])

In [8]:
def salvar(status,tipo,newLine):
    f = open('DOWNLOAD/PROCESSADO/processamento_' + status + "." + tipo ,"a+")
    f.write(newLine.encode('utf-8') + '\n')
    f.close()

In [9]:
executar()

KeyboardInterrupt: 